# 面試趣 Word2Vec - 算字詞間相似度

In [1]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
from gensim import models
import ast


In [90]:
df = pd.read_csv('./data/toyota/case01_toyota.csv', sep=',', header=0)
df

,system_id,result
0,1,"['運動', '休旅車', '正夯', '熱鬧', '車長', 'mm', '區塊', '國..."
1,2,"['toyota', 'cc', '年式', '油電', '豪華', '另計', '需綁',..."
2,3,"['打破', '車壇', '史上', '單月', '銷售', '台數', '銷量', '超過..."
3,4,"['預計', '明年', '現身', 'toyota', 'rav', '將推', '動力'..."
4,5,"['新一代', 'mazda', '有望', '沿用', 'yaris', '平台', 'c..."
...,...,...
10610,10611,"['認為', '逼車', '行駛', '右側', '金門', '鄉親', '小心', '銀色..."
10611,10612,"['陣子', '機車', 'altis', '交通', '號誌', '十字路口', '相撞'..."
10612,10613,"['事情', '那天', '騎著', '機車', '路邊', '逆向', '對面', '轉格..."
10613,10614,"['描述', '事發', '過程', '忘記', '提到', '重要', '事情', '車禍..."


In [91]:
df['result'] = df['result'].apply(lambda x: ast.literal_eval(x))
df['result'] = df['result'].apply(lambda x: ' '.join(x))
df

,system_id,result
0,1,運動 休旅車 正夯 熱鬧 車長 mm 區塊 國產車 toyota corolla cros ...
1,2,toyota cc 年式 油電 豪華 另計 需綁 乙式 實報實銷 另計 實報實銷 成交 價格...
2,3,打破 車壇 史上 單月 銷售 台數 銷量 超過 納智捷 整年 車系 目標 證明 台灣人 越來...
3,4,預計 明年 現身 toyota rav 將推 動力 節能 碳排放 政策 豐田 集團 終於 加...
4,5,新一代 mazda 有望 沿用 yaris 平台 cx 辦理 有待 查證 內文 提到 歐洲 ...
...,...,...
10610,10611,認為 逼車 行駛 右側 金門 鄉親 小心 銀色 toyota corona 車牌 今晚 環島...
10611,10612,陣子 機車 altis 交通 號誌 十字路口 相撞 肇事 責任 示意圖 imgur 藍色 黃...
10612,10613,事情 那天 騎著 機車 路邊 逆向 對面 轉格 待轉 白目 那天 發什麼 神經 逆向 突然 ...
10613,10614,描述 事發 過程 忘記 提到 重要 事情 車禍 受傷 發生 車禍 事故 發生 騎車 機車道 ...


In [92]:
output = open('./data/toyota/toyota_w2v.txt', 'w')
for index, row in df.iterrows():
    # tmp = row['result'].split(",")
    output.write(row['result'] + '\n')

## 實作 word2vec

訓練 model

In [63]:
sentences = word2vec.LineSentence("./data/ford/ford_w2v.txt")
model = word2vec.Word2Vec(sentences, sg=1, window=6, min_count=30, workers=6, vector_size=250)

# 保存模型，供日後使用
model.save("./data/ford/word2vec.model")

# 模型讀取方式
# model = word2vec.Word2Vec.load("your_model_name")

## 根據 word 前 n 個相關詞

load model

In [66]:
from gensim import models
model = models.Word2Vec.load('./data/ford/word2vec.model')

測試

In [78]:
print("輸入一個詞，則去尋找前十個該詞的相似詞")           
query = "福特"         
print("相似詞前 10 排序")                
res = model.wv.most_similar(query, topn=10)                
for item in res:                    
    print(item[0] + "," + str(item[1]))

輸入一個詞，則去尋找前十個該詞的相似詞
相似詞前 10 排序
活得,0.5614586472511292
修得,0.5458086133003235
福特車,0.5350620150566101
痛快,0.5283123850822449
買過,0.5255555510520935
精采,0.5228757262229919
nford,0.5218576788902283
如題,0.5177111625671387
六和,0.5162533521652222
主修,0.5140368342399597


定意函式：給一個 word 回傳前 n 個最相關詞

In [68]:
def get_top6_similar_word(word):
    res = model.wv.most_similar(word, topn=10)
    return res

根據詞頻高的字去找

In [70]:
df_aspect = pd.read_csv('./data/ford/case01_ford_freq.csv', sep=',', header=0)
df_aspect = df_aspect.head(200)
df_aspect

,word,freq
0,福特,7142
1,問題,4905
2,業務,3825
3,原廠,3742
4,Focus,2984
...,...,...
195,以下,601
196,12,600
197,11,599
198,兩個,598


In [74]:
df_most_correlation = pd.DataFrame(columns=['item1', 'item2', 'correlation'])

In [75]:
for index, row in df_aspect.iterrows():
    try:
        top6 = get_top6_similar_word(row['word'])
    except:
        print(row['word'])
        continue
    for item in top6:
        tmp = {'item1': row['word'], 'item2': item[0], 'correlation': str(item[1])}
        df_most_correlation = df_most_correlation.append(tmp, ignore_index = True)

Focus
Ford
Kuga
CC
Toyota
15
ST
Active
KUGA
CRV
RAV4
30
ACC
SUV
2021
100
180
QQ
12
11


In [76]:
df_most_correlation

,item1,item2,correlation
0,福特,活得,0.5614586472511292
1,福特,修得,0.5458086133003235
2,福特,福特車,0.5350620150566101
3,福特,痛快,0.5283123850822449
4,福特,買過,0.5255555510520935
...,...,...,...
1795,影響,減少,0.5463711619377136
1796,影響,車用,0.5366078019142151
1797,影響,不佳,0.5202406048774719
1798,影響,供應,0.5129908323287964


In [77]:
df_most_correlation.to_csv('./data/ford/ford_correlation.csv', encoding='utf-8')

跟據構面去找

In [ ]:
薪水
特休
三節獎金
年終
加班
輪班
性能
馬力
動力
外觀
操控
安全
堅固
舒適
價錢
科技
品質
抽獎
好禮
徵才
實習
工作
人資
離職
轉職
新鮮人
畢業
出路
電資
機械
製造
車輛工程
品管
傳產